# Load Trajectory

In [1]:
from evo.core.trajectory import PosePath3D, PoseTrajectory3D
from scipy.spatial.transform import Rotation as Rot
import numpy as np
import dill

dumpfile = open("../notebooks/Dataset_Syntetic/Sets/1/gt.dill", "rb")
trajectory: PoseTrajectory3D = dill.load(dumpfile)
dumpfile.close()

# Calculate Drift

In [2]:
# grep poses and timestamps
poses = trajectory.poses_se3
timestamps = trajectory.timestamps


# Initialize cumulative drift in local frame
cumulative_drift_local = np.zeros(6)  # Drift for [x, y, z, roll, pitch, yaw] in the robot's local frame

# Initialize list for poses with drift
poses_with_drift = [poses[0]]  # Start with the first pose

# Define big drift matrix (K) with all elements explicitly written
K = np.array([
    [-0.5,  0.0,  0.0,  0.0,  0.0,  0.0],  # x drift due to x, y, z, roll, pitch, yaw
    [0.5,  0.0,   0.0,  0.0,  0.0,  0.0],  # y drift due to x, y, z, roll, pitch, yaw
    [ 0.1,  0.0,  0.0,  0.0,  0.0,  0.0],  # z drift due to x, y, z, roll, pitch, yaw
    [ 0.0,  0.0,  0.0,  0.0,  0.0,  0.0],  # roll drift due to x, y, z, roll, pitch, yaw
    [ 0.0,  0.0,  0.0,  0.0,  0.0,  0.0],  # pitch drift due to x, y, z, roll, pitch, yaw
    [ 0.0,  0.0,  0.0,  0.0,  0.0,  0.0]  # yaw drift due to x, y, z, roll, pitch, yaw
])

# Define noise covariance matrix (Q) for stochastic noise
Q = np.diag([0.0001, 0.0001, 0.0001, 0.00001, 0.00001, 0.00001])  # Noise for [x, y, z, roll, pitch, yaw]

In [3]:
# Loop through all poses to add systematic errors
for i in range(1, len(poses)):
    dt = timestamps[i] - timestamps[i - 1]
    if dt > 0:
        # Extract current and previous poses
        prev_pose = poses[i - 1]
        curr_pose = poses[i]

        # Compute relative motion in the global frame
        relative_translation = curr_pose[:3, 3] - prev_pose[:3, 3]
        R_prev = Rot.from_matrix(prev_pose[:3, :3])  # Orientation of the previous pose

        # Transform relative translation to the local frame of the robot
        relative_translation_local = R_prev.inv().apply(relative_translation)

        # Compute relative angular motion
        R_curr = Rot.from_matrix(curr_pose[:3, :3])  # Orientation of the current pose
        relative_rotation = R_curr * R_prev.inv()
        relative_euler_local = relative_rotation.as_euler('xyz')  # Relative angles in local frame

        # Combine relative motion into a single vector
        relative_motion_local = np.hstack([relative_translation_local, relative_euler_local])

        # Apply systematic drift using the big K matrix
        drift_increment = K @ relative_motion_local * dt

        # Update cumulative drift
        cumulative_drift_local += drift_increment

        # Transform drift back to the global frame
        linear_drift_global = cumulative_drift_local[:3] #R_prev.apply(cumulative_drift_local[:3])  # Transform linear drift
        angular_drift_local = cumulative_drift_local[3:]  # Angular drift remains in local frame

        # Apply the drift to the current pose
        new_pose = curr_pose.copy()
        new_pose[:3, 3] += linear_drift_global  # Add linear drift to the global translation

        # Apply angular drift in the local frame
        current_euler_global = Rot.from_matrix(new_pose[:3, :3]).as_euler('xyz')
        new_euler_global = current_euler_global + angular_drift_local
        new_pose[:3, :3] = Rot.from_euler('xyz', new_euler_global).as_matrix()

        poses_with_drift.append(new_pose)
    else:
        poses_with_drift.append(curr_pose)  # Copy pose without changes for zero-time difference

# Convert back to PoseTrajectory3D
trajectory_with_drift = PoseTrajectory3D(
    timestamps=timestamps,
    poses_se3=poses_with_drift
)


In [4]:
from mathUtils import calculate_yaw_from_points, calculate_pitch_from_points

# Extract positions (assuming SE(3) poses)
x = np.array([pose[0, 3] for pose in trajectory_with_drift.poses_se3])
y = np.array([pose[1, 3] for pose in trajectory_with_drift.poses_se3])
z = np.array([pose[2, 3] for pose in trajectory_with_drift.poses_se3])

# Calculate new yaw and pitch
new_yaw = calculate_yaw_from_points(x, y)
new_pitch = calculate_pitch_from_points(x, y, z)

# Update poses in the trajectory
for i, pose in enumerate(trajectory_with_drift.poses_se3):
    rotation_matrix = pose[:3, :3]
    # Decompose the rotation matrix
    roll, _, _ = Rot.from_matrix(new_pose[:3, :3]).as_euler('xyz')
    # Create updated rotation matrix
    new_rotation = Rot.from_euler('xyz', [roll, new_pitch[i], new_yaw[i]], degrees=True).as_matrix()
    # Update the SE(3) matrix
    pose[:3, :3] = new_rotation


In [5]:
for i, pose in enumerate(trajectory_with_drift.poses_se3):   
    # Add stochastic noise using the Q matrix
    noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q)

    #add translational noise
    pose[:3, 3] += noise[:3]

    #add angular noise
    current_euler_global = Rot.from_matrix(pose[:3, :3]).as_euler('xyz')
    new_euler_global = current_euler_global + angular_drift_local
    pose[:3, :3] = Rot.from_euler('xyz', new_euler_global).as_matrix()





In [6]:


trajectory_with_drift.meta["Name"] = "odom"
trajectory_with_drift.meta["topic"] = "/localization/odometry/odom_lidar"
trajectory_with_drift.meta["frame_id"] = "odom"
trajectory_with_drift.meta["child_frame_id"] = "base_link"
trajectory_with_drift.meta["publish_transform"] = True
trajectory_with_drift.meta["publish_orientation"] = False
trajectory_with_drift.meta["frequency"] = 10

# Save the new trajectory
output_file = "../notebooks/Dataset_Syntetic/Sets/1/odom_large_drift.dill"
with open(output_file, "wb") as f:
    dill.dump(trajectory_with_drift, f)

print(f"Drifted trajectory with linear and angular drift (per-axis) saved to {output_file}")

trajectory_with_drift.meta["frequency"] = 5

# Save the new trajectory
output_file = "../notebooks/Dataset_Syntetic/Sets/1/odom_larg_drift_half_speed.dill"
with open(output_file, "wb") as f:
    dill.dump(trajectory_with_drift, f)

print(f"Drifted trajectory with linear and angular drift (per-axis) saved to {output_file}")

Drifted trajectory with linear and angular drift (per-axis) saved to ../notebooks/Dataset_Syntetic/Sets/1/odom_large_drift.dill
Drifted trajectory with linear and angular drift (per-axis) saved to ../notebooks/Dataset_Syntetic/Sets/1/odom_larg_drift_half_speed.dill
